# Construct X and y Matrices

In previous work, we have extracted data for 300 top returning stocks over the last month. Of these, 30 were selected for further study. On these 30, we implemented sinusoidal features on Open, High, Low, and Close positions. Using the close, we also algorithmically labeled buy, sell, and hold points as 1, 2, and 0 respectively. 

In order to train a model on these datasets, we must now ensure they are correctly constructed. The model we will be using will be a Convolutional Neural Network, so our X matrix must be constructed from 'images' comprised of the featurres. Our y will be an array. 

Since the goal will be to train a single, generalized model, our X and y dataset should contain shuffled data from the first half of our time period. Data from the second half accross all stocks will be held back for test data. 

Each dataset will first be split into train and test. Y will then be removed into an array of its own. The remaining X features will need to be transformed to comprise images. Both of these X and y sections will be deposited into the larger X/y matrices. 

#### Steps

1. Iteratively load the datasets
2. On each iteration,   
    A. Drop unnecessary columns  
    B. Sterile the data by scaling relative columns (open, high, low, close, volume) and those columns with very small values
    C. Remove y data and store in y Matrix  
    D. Pivot the X data  
    E. Slice the X data and store it in an array  
4. X.shape should equal (30, 8000, 30, 30) and y.shape (30, 8000)  
5. Save the base X and y  
6. Iterate over the elements of X and y together:  
    A. Split by second index (time)  
    B. Reshape data into train X (120,000, 30, 30) train y (120,000) test X (120,000, 30, 30) test y (120,000)  
    C. Save train and test data sets  
7. Use preprocessing functions:  
    A. balance classes 33/33/33  
    B. Shuffle the data (possibly)  
8. Save   

### 1. & 2. Iteratively load and transform the datsets

In [1]:
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
from extract import load_set
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler

pd.set_option('display.max_columns', 130)
pd.set_option('display.max_rows', 100)

In [2]:
data_dir = './data/prepared/august25screenfixed/'
suffix = ''

stocks = [f.split('.')[0] for f in listdir(data_dir) if isfile(join(data_dir, f))]
stocks

['RRR',
 'RLGY',
 'HOME',
 'FDX',
 'MIK',
 'WSM',
 'NVDA',
 'DE',
 'TGT',
 'LULU',
 'FBHS',
 'XRX',
 'CX',
 'EAT',
 'DHI',
 'ICE',
 'EXPI',
 'CLNY',
 'IMVT',
 'ELAN',
 'PACB',
 'PENN',
 'REAL',
 'NKE',
 'BBY',
 'HTHT',
 'GRPN',
 'CZR',
 'GDDY',
 'LOW']

In [3]:
df = load_set(stocks[0], data_dir, suffix)
df.head()

,open,high,low,close,volume,datetime,date,hour,minute,min_num,SYMBOL,prev_close,diff_1,pct_change,log_return,%open,mesa_open,open_amp,open_omega,open_phase,open_offset,open_freq,open_period,open_maxcov,open_sin,time,open_angle,open_rad,open_rad2,open_sin2,open_cos2,open_cos,open_tan,open_tan2,open_xsinx,open_xcosx,open_sinxcosx,open_xsinxcosx,open_xsinx2,open_xcosx2,open_sinxcosx2,open_xsinxcosx2,open_xtanx,open_xtanx2,%high,mesa_high,high_amp,high_omega,high_phase,high_offset,high_freq,high_period,high_maxcov,high_sin,high_angle,high_rad,high_rad2,high_sin2,high_cos2,high_cos,high_tan,high_tan2,high_xsinx,high_xcosx,high_sinxcosx,high_xsinxcosx,high_xsinx2,high_xcosx2,high_sinxcosx2,high_xsinxcosx2,high_xtanx,high_xtanx2,%low,mesa_low,low_amp,low_omega,low_phase,low_offset,low_freq,low_period,low_maxcov,low_sin,low_angle,low_rad,low_rad2,low_sin2,low_cos2,low_cos,low_tan,low_tan2,low_xsinx,low_xcosx,low_sinxcosx,low_xsinxcosx,low_xsinx2,low_xcosx2,low_sinxcosx2,low_xsinxcosx2,low_xtanx,low_xtanx2,%close,mesa_close,close_amp,close_omega,close_phase,close_offset,close_freq,close_period,close_maxcov,close_sin,close_angle,close_rad,close_rad2,close_sin2,close_cos2,close_cos,close_tan,close_tan2,close_xsinx,close_xcosx,close_sinxcosx,close_xsinxcosx,close_xsinx2,close_xcosx2,close_sinxcosx2,close_xsinxcosx2,close_xtanx,close_xtanx2,decision,D2
8845,17.28,17.28,17.28,17.28,100,2020-08-25 22:48:00,2020-08-25,22,48,1368,RRR,17.30,-0.02,-0.001156,-0.001157,-0.001156,0.490588,0.001894,2.999950,141.151213,-0.000400,0.477457,2.094430,2.221737e+05,-0.001211,8845,26675.705720,3.584091,4.061547,-0.001907,-0.605856,-0.903685,0.473836,1.313140,0.000001,0.001045,0.001095,-0.000001,2.204898e-06,0.000700,0.001156,-1.335851e-06,-0.000548,-0.001518,-0.001156,0.490591,-0.001940,2.802241,-332.993391,-0.000401,0.445991,2.242200,2.440428e+05,0.001482,24452.828207,4.954177,5.400168,0.001098,0.634822,0.239439,-4.054941,-1.217125,-0.000002,-0.000277,0.000355,-4.103498e-07,-0.000001,-0.000734,0.000697,-8.057384e-07,0.004688,0.001407,-0.001156,0.490588,0.002575,2.990564,224.104713,-0.000389,0.475963,2.101003,1.440349e+05,-0.001352,26675.646515,3.524886,4.000850,-0.002340,-0.653000,-0.927438,0.403237,1.159812,0.000002,0.001072,0.001254,-0.000001,0.000003,0.000755,0.001528,-0.000002,-0.000466,-0.001341,-0.001156,0.490591,-0.002573,2.833824,-611.936535,-0.000390,0.451017,2.217211,1.726659e+05,0.001653,24453.239631,5.365601,5.816618,0.000767,0.893118,0.607740,-1.306702,-0.503655,-0.000002,-0.000703,0.001004,-0.000001,-8.865898e-07,-0.001033,0.000685,-7.918291e-07,0.001511,0.000582,0.0,0.0
8844,17.30,17.30,17.30,17.30,200,2020-08-25 22:44:00,2020-08-25,22,44,1364,RRR,17.44,-0.14,-0.008028,-0.008060,-0.002882,0.489934,0.001881,1.287313,-270.196033,-0.000346,0.204882,4.880852,2.322576e+05,-0.000633,8844,11114.801657,6.130034,6.334916,-0.000249,0.998662,0.988295,-0.154360,0.051777,0.000002,-0.002848,-0.000626,0.000002,7.176933e-07,-0.002878,-0.000249,7.167332e-07,0.000445,-0.000149,-0.008028,0.488493,-0.002108,2.808354,-387.061715,-0.000335,0.446963,2.237320,1.819360e+05,-0.002103,24450.020219,2.146189,2.593152,-0.001434,-0.853339,-0.544164,-1.541776,-0.610962,0.000017,0.004368,0.001145,-9.187537e-06,0.000012,0.006850,0.001223,-9.820883e-06,0.012377,0.004905,-0.002882,0.489934,0.002457,3.001820,124.667322,-0.000327,0.477755,2.093125,2.107880e+05,0.001151,26672.767225,0.645596,1.123351,0.001888,0.432664,0.798741,0.753279,2.083731,-0.000003,-0.002302,0.000919,-0.000003,-0.000005,-0.001247,0.000817,-0.000002,-0.002171,-0.006005,-0.008028,0.488493,-0.002719,2.838034,-649.214703,-0.000325,0.451687,2.213922,1.408935e+05,-0.001990,24450.356426,2.482395,2.934083,-0.000885,-0.978547,-0.790484,-0.774819,-0.210541,0.000016,0.006346,0.001573,-0.000013,7.103888e-06,0.007855,0.000866,-6.951488e-06,0.006220,0.001690,0.0,0.0
8843,17.35,17.44,17.35,17.44,25,2020-08-25 22:36:00,2020-08-25,22,36,1356,RRR,17.35,0.09,0.005187,0.005174,0.000000,0.496402,0.004198,3.106833,-802.280898,-0

In [4]:
df.dropna(axis=0).shape
#df.shape

(8819, 129)

In [5]:
df.describe()

,open,high,low,close,volume,hour,minute,min_num,prev_close,diff_1,pct_change,log_return,%open,mesa_open,open_amp,open_omega,open_phase,open_offset,open_freq,open_period,open_maxcov,open_sin,time,open_angle,open_rad,open_rad2,open_sin2,open_cos2,open_cos,open_tan,open_tan2,open_xsinx,open_xcosx,open_sinxcosx,open_xsinxcosx,open_xsinx2,open_xcosx2,open_sinxcosx2,open_xsinxcosx2,open_xtanx,open_xtanx2,%high,mesa_high,high_amp,high_omega,high_phase,high_offset,high_freq,high_period,high_maxcov,high_sin,high_angle,high_rad,high_rad2,high_sin2,high_cos2,high_cos,high_tan,high_tan2,high_xsinx,high_xcosx,high_sinxcosx,high_xsinxcosx,high_xsinx2,high_xcosx2,high_sinxcosx2,high_xsinxcosx2,high_xtanx,high_xtanx2,%low,mesa_low,low_amp,low_omega,low_phase,low_offset,low_freq,low_period,low_maxcov,low_sin,low_angle,low_rad,low_rad2,low_sin2,low_cos2,low_cos,low_tan,low_tan2,low_xsinx,low_xcosx,low_sinxcosx,low_xsinxcosx,low_xsinx2,low_xcosx2,low_sinxcosx2,low_xsinxcosx2,low_xtanx,low_xtanx2,%close,mesa_close,close_amp,close_omega,close_phase,close_offset,close_freq,close_period,close_maxcov,close_sin,close_angle,close_rad,close_rad2,close_sin2,close_cos2,close_cos,close_tan,close_tan2,close_xsinx,close_xcosx,close_sinxcosx,close_xsinxcosx,close_xsinx2,close_xcosx2,close_sinxcosx2,close_xsinxcosx2,close_xtanx,close_xtanx2,decision
count,8846.000000,8846.000000,8846.000000,8846.000000,8846.000000,8846.000000,8846.000000,8846.000000,8843.000000,8843.000000,8843.000000,8843.000000,8845.000000,8842.000000,8821.000000,8821.000000,8821.000000,8821.000000,8821.000000,8821.000000,8.821000e+03,8821.000000,8846.000000,8821.000000,8821.000000,8821.000000,8821.000000,8821.000000,8821.000000,8821.000000,8821.000000,8.821000e+03,8821.000000,8821.000000,8.821000e+03,8.821000e+03,8821.000000,8821.000000,8.821000e+03,8821.000000,8821.000000,8845.000000,8842.000000,8821.000000,8821.000000,8821.000000,8821.000000,8821.000000,8821.000000,8.821000e+03,8821.000000,8821.000000,8821.000000,8821.000000,8821.000000,8821.000000,8821.000000,8821.000000,8821.000000,8.821000e+03,8821.000000,8821.000000,8.821000e+03,8.821000e+03,8821.000000,8821.000000,8.821000e+03,8821.000000,8821.000000,8845.000000,8842.000000,8821.000000,8821.000000,8821.000000,8821.000000,8821.000000,8821.000000,8.821000e+03,8821.000000,8821.000000,8821.000000,8821.000000,8821.000000,8821.000000,8821.000000,8821.000000,8821.000000,8.821000e+03,8821.000000,8821.000000,8.821000e+03,8.821000e+03,8821.000000,8821.000000,8.821000e+03,8821.000000,8821.000000,8845.000000,8842.000000,8821.000000,8821.000000,8821.000000,8821.000000,8821.000000,8821.000000,8.821000e+03,8821.000000,8821.000000,8821.000000,8821.000000,8821.000000,8821.000000,8821.000000,8.821000e+03,8821.000000,8.821000e+03,8821.000000,8821.000000,8.821000e+03,8.821000e+03,8821.000000,8821.000000,8.821000e+03,8821.000000,8821.000000,8846.000000
mean,14.145617,14.159394,14.132140,14.146278,5692.403572,16.267579,30.398485,1006.453199,14.145844,0.000747,0.000058,0.000054,0.000055,0.491104,0.003151,1.580050,-3.909109,0.003543,0.251473,17.961551,inf,0.000063,4422.500000,6870.890667,3.196750,3.448223,0.000336,0.011294,0.011014,1.517650,-0.200557,2.675746e-06,0.000013,-0.000002,-4.342282e-08,-1.062358e-05,-0.000015,0.000071,3.249090e-06,-0.003242,0.000091,0.000055,0.491103,0.001029,1.460280,-2.132063,0.001876,0.232411,19.753840,inf,0.000062,6619.616764,3.178317,3.410728,-0.000301,0.007198,0.007982,0.476751,-4.940400,2.483268e-06,-0.000003,-0.000008,4.126265e-08,-1.071168e-05,-0.000055,-0.000446,-1.524439e-06,-0.007622,-0.002061,0.000055,0.491104,0.001177,1.531006,11.975717,-0.000067,0.243667,19.265260,inf,0.000068,6654.423104,3.171779,3.415446,0.000099,-0.000909,-0.001330,-0.795871,0.907205,2.639368e-06,0.000017,0.000015,-9.723026e-08,-6.009884e-06,-0.000032,0.000074,3.344522e-06,-0.004453,0.005429,0.000056,0.491104,0.001226,1.665249,6.570902,0.000950,0.265033,16.457720,inf,0.000072,7347.135854,3.200566,3.465599,0.000079,0.008360,0.006554,-1.789241e+02,

In [6]:
df['open_maxcov'].values

array([ 222173.70242215,  232257.56696686, 6295939.83291142, ...,
                    nan,              nan,              nan])

In [18]:
print('\n'.join(df['open_maxcov'].values.astype('str')))

222173.70242215143
232257.5669668576
6295939.832911415
2372299301.0622935
17649813964.26809
2129393173.6066098
18168633068.746582
256633.3589228578
257406.38033527767
319907.6702010652
421745.5174003669
317201.93217751785
354169.7504004168
339895.80418645317
275317.0087003242
276704.8478375882
285092.9818454513
238259.84282341323
202470.21998356818
263346.97576999885
235493.1043254767
214877.81802646848
223583.57977778921
223998.340193155
258491.93987983337
248069.2786508555
241830.10467642112
290883.8911386318
263929.04789361585
258447.0318921178
299127.4692672525
336737.12757972925
348780.1474519252
297060.97912904073
344396.9640455302
227495.14325655677
158120.58103435655
221468485.55281866
2269273.425141904
297283.9834251032
205895.25310121244
122492.42933156829
184619.86277110453
269596.2717459314
225142.8366733965
211191.48468244806
213588.94593907314
2394245.49947217
737699.1005373549
234440.15415089423
2649945718.902009
308004.02474493516
247635.83685140798
282807.8276968841
40

#### A. Drop unnecessary columns

In [45]:
df.head(5)

,open,high,low,close,volume,datetime,date,hour,minute,min_num,SYMBOL,prev_close,diff_1,pct_change,log_return,%open,mesa_open,open_amp,open_omega,open_phase,open_offset,open_freq,open_period,open_maxcov,open_sin,time,open_angle,open_rad,open_rad2,open_sin2,open_cos2,open_cos,open_tan,open_tan2,open_xsinx,open_xcosx,open_sinxcosx,open_xsinxcosx,open_xsinx2,open_xcosx2,open_sinxcosx2,open_xsinxcosx2,open_xtanx,open_xtanx2,%high,mesa_high,high_amp,high_omega,high_phase,high_offset,high_freq,high_period,high_maxcov,high_sin,high_angle,high_rad,high_rad2,high_sin2,high_cos2,high_cos,high_tan,high_tan2,high_xsinx,high_xcosx,high_sinxcosx,high_xsinxcosx,high_xsinx2,high_xcosx2,high_sinxcosx2,high_xsinxcosx2,high_xtanx,high_xtanx2,%low,mesa_low,low_amp,low_omega,low_phase,low_offset,low_freq,low_period,low_maxcov,low_sin,low_angle,low_rad,low_rad2,low_sin2,low_cos2,low_cos,low_tan,low_tan2,low_xsinx,low_xcosx,low_sinxcosx,low_xsinxcosx,low_xsinx2,low_xcosx2,low_sinxcosx2,low_xsinxcosx2,low_xtanx,low_xtanx2,%close,mesa_close,close_amp,close_omega,close_phase,close_offset,close_freq,close_period,close_maxcov,close_sin,close_angle,close_rad,close_rad2,close_sin2,close_cos2,close_cos,close_tan,close_tan2,close_xsinx,close_xcosx,close_sinxcosx,close_xsinxcosx,close_xsinx2,close_xcosx2,close_sinxcosx2,close_xsinxcosx2,close_xtanx,close_xtanx2,decision
8845,17.28,17.28,17.28,17.28,100,2020-08-25 22:48:00,2020-08-25,22,48,1368,RRR,17.30,-0.02,-0.001156,-0.001157,-0.001156,0.490588,0.001894,2.999950,141.151213,-0.000400,0.477457,2.094430,2.221737e+05,-0.001211,8845,26675.705720,3.584091,4.061547,-0.001907,-0.605856,-0.903685,0.473836,1.313140,0.000001,0.001045,0.001095,-0.000001,2.204898e-06,0.000700,0.001156,-1.335851e-06,-0.000548,-0.001518,-0.001156,0.490591,-0.001940,2.802241,-332.993391,-0.000401,0.445991,2.242200,2.440428e+05,0.001482,24452.828207,4.954177,5.400168,0.001098,0.634822,0.239439,-4.054941,-1.217125,-0.000002,-0.000277,0.000355,-4.103498e-07,-0.000001,-0.000734,0.000697,-8.057384e-07,0.004688,0.001407,-0.001156,0.490588,0.002575,2.990564,224.104713,-0.000389,0.475963,2.101003,1.440349e+05,-0.001352,26675.646515,3.524886,4.000850,-0.002340,-0.653000,-0.927438,0.403237,1.159812,0.000002,0.001072,0.001254,-0.000001,0.000003,0.000755,0.001528,-0.000002,-0.000466,-0.001341,-0.001156,0.490591,-0.002573,2.833824,-611.936535,-0.000390,0.451017,2.217211,1.726659e+05,0.001653,24453.239631,5.365601,5.816618,0.000767,0.893118,0.607740,-1.306702,-0.503655,-0.000002,-0.000703,0.001004,-0.000001,-8.865898e-07,-0.001033,0.000685,-7.918291e-07,0.001511,0.000582,0.0
8844,17.30,17.30,17.30,17.30,200,2020-08-25 22:44:00,2020-08-25,22,44,1364,RRR,17.44,-0.14,-0.008028,-0.008060,-0.002882,0.489934,0.001881,1.287313,-270.196033,-0.000346,0.204882,4.880852,2.322576e+05,-0.000633,8844,11114.801657,6.130034,6.334916,-0.000249,0.998662,0.988295,-0.154360,0.051777,0.000002,-0.002848,-0.000626,0.000002,7.176933e-07,-0.002878,-0.000249,7.167332e-07,0.000445,-0.000149,-0.008028,0.488493,-0.002108,2.808354,-387.061715,-0.000335,0.446963,2.237320,1.819360e+05,-0.002103,24450.020219,2.146189,2.593152,-0.001434,-0.853339,-0.544164,-1.541776,-0.610962,0.000017,0.004368,0.001145,-9.187537e-06,0.000012,0.006850,0.001223,-9.820883e-06,0.012377,0.004905,-0.002882,0.489934,0.002457,3.001820,124.667322,-0.000327,0.477755,2.093125,2.107880e+05,0.001151,26672.767225,0.645596,1.123351,0.001888,0.432664,0.798741,0.753279,2.083731,-0.000003,-0.002302,0.000919,-0.000003,-0.000005,-0.001247,0.000817,-0.000002,-0.002171,-0.006005,-0.008028,0.488493,-0.002719,2.838034,-649.214703,-0.000325,0.451687,2.213922,1.408935e+05,-0.001990,24450.356426,2.482395,2.934083,-0.000885,-0.978547,-0.790484,-0.774819,-0.210541,0.000016,0.006346,0.001573,-0.000013,7.103888e-06,0.007855,0.000866,-6.951488e-06,0.006220,0.001690,0.0
8843,17.35,17.44,17.35,17.44,25,2020-08-25 22:36:00,2020-08-25,22,36,1356,RRR,17.35,0.09,0.005187,0.005174,0.000000,0.496402,0.004198,3.106833,-802.280898,-0.000187,0.4

In [7]:
df['open'] = df['open'] - df['open'][::-1].rolling(30).mean()[::-1]

In [5]:

drop_cols = ['datetime',
            'date',
            'min_num',
            'SYMBOL',
            'prev_close',
            'diff_1',
            'time',
            'decision',
            'open_maxcov',
            'high_maxcov',
            'low_maxcov',
            'close_maxcov',
            'pct_change',
            'D2',]


df.drop(drop_cols, axis=1, inplace=True)
df.shape

(8846, 116)

In [6]:
X = np.zeros([30,4000,116,60])
y = np.zeros([30,4000])

drop_cols = ['datetime',
            'date',
            'min_num',
            'SYMBOL',
            'prev_close',
            'diff_1',
            'time',
            'decision',
            'open_maxcov',
            'high_maxcov',
            'low_maxcov',
            'close_maxcov',
            'pct_change',
            'D2',
            'target']

for k, stock in enumerate(stocks):
    df = load_set(stock, data_dir, suffix)
    
    for col in ['open','high','low','close']:
        df[col] = df[col] - df[col][::-1].rolling(30).mean()[::-1]
        
    df = df.dropna(axis=0)
    
    e = df.shape[0]
    l = e - 4000 - 59

    df['target'] = df['%close'].shift(1)
    y_ = df.iloc[l:e-59]['target']
    y[k] = y_.to_numpy() 

    df.drop(drop_cols, axis=1, inplace=True)
    
    
    
    ## TO DO: - standardscale Open, High, Low, Close columns
    ##        - consider minmax scaling trig columns where values are very close to zero (either upon inspection or universally)
    ##        - many of the interaction terms are very small because %min change is very small
    ##.       - either simply scale the interaction terms after creation
    ##.       - Or consider scaling the %min change columns prior to creating interaction terms 
    
    ##        NOTE: Originally I decided on standardizing the features. However, later in the process the features 
    ##              **MUST** be normalized between 0 and 1 for image recognition. 
    ##              Because of this contradiction, I have commented out the standardization code and instead
    ##.             implemented normalization on 0 - 1 from the beginning. 
    
#     ss = StandardScaler()

#     ss.fit(df.iloc[l:e-59-2000])
#     scaled_features = ss.transform(df)
#     df = pd.DataFrame(scaled_features, index=df.index, columns=df.columns)

    mx = MinMaxScaler()
    mx = mx.fit(df)
    scaled_features = mx.transform(df)
    df = pd.DataFrame(scaled_features, index=df.index, columns=df.columns)

    dt = df.transpose()

    for j, i in enumerate(range(l, e-59)):
        X[k][j] = dt.iloc[:, i:i+60].to_numpy()
        
np.save('./data/prepared/august25screenfixed/numpy_matrices/X_f.npy', X)
np.save('./data/prepared/august25screenfixed/numpy_matrices/y_f.npy', y)

In [2]:
X = np.load('./data/prepared/august25screenfixed/numpy_matrices/X.npy')
y = np.load('./data/prepared/august25screenfixed/numpy_matrices/y.npy')

In [3]:
X.shape, y.shape

((30, 4000, 116, 60), (30, 4000))

Unfortunately, the X file is 7 GB, which is prohibitively large. When working with this much data, I suppose it is expected to be large. 

In [6]:
zeros = np.where(X == 0)
zeros

(array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64))

In [7]:
X[0][0]

array([[-0.14538124,  0.15174104,  0.20069869, ...,  0.64655092,
         2.5136943 ,  2.81844721],
       [ 0.17827578, -0.0225963 ,  0.01856601, ...,  0.26926095,
         2.04253328,  2.78431104],
       [-0.18166262,  0.16911168,  0.36663506, ..., -0.70908412,
         1.01754274,  2.79954426],
       ...,
       [ 0.01076609,  0.01083497,  0.01071884, ...,  0.03321744,
        -0.07488926,  0.01197654],
       [ 0.01780772,  0.01655196,  0.01597589, ...,  0.21659399,
         0.06561708,  0.01615344],
       [-0.01091948, -0.02560259, -0.0490432 , ...,  3.53065162,
         0.44677296, -0.03398237]])

#### Reshape data into train X (60,000, 30, 30) train y (60,000) test X (60,000, 30, 30) test y (60,000)

Iterate over the X and y datasets. For each stock, split into train and test by time (:2000, 2000:). Save as X_train, X_test, y_train, and y_test. 

In [1]:
import numpy as np

In [2]:
X = np.load('./data/prepared/august25screenfixed/numpy_matrices/X.npy')

In [3]:
X_train = X[:, :2000, :, :]

In [4]:
del(X)

In [ ]:
X_train.shape = (60000, 116, 60, 1)

In [ ]:
X_train = X[:, :2000, :, :]
del(X)
X_ = X_train.reshape([60000, 116, 60, 1])
np.save('./data/prepared/august25screenfixed/numpy_matrices/X_train.npy', X)

In [ ]:
X_train = X[:, :2000, :, :].reshape([60000, 116, 60, 1])
np.save('./data/prepared/august25screenfixed/numpy_matrices/X_train.npy', X_train)
del(X_train)

In [ ]:
X_test = X[:, 2000:, :, :].reshape([60000, 116, 60, 1])
np.save('./data/prepared/august25screenfixed/numpy_matrices/X_test.npy', X_test)
del(X_test)

In [12]:
y_train = y[:, :2000].reshape([60000, 1])
np.save('./data/prepared/august25screenfixed/numpy_matrices/y_train.npy', y_train)

In [13]:
y_test = y[:, 2000:].reshape([60000, 1])
np.save('./data/prepared/august25screenfixed/numpy_matrices/y_test.npy', y_test)

In [11]:
np.unique(y_train, return_counts=True), np.unique(y_test,return_counts=True)

((array([0., 1., 2.]), array([34245, 16007,  9748])),
 (array([0., 1., 2.]), array([34719, 14301, 10980])))

In [14]:
y_train[:10]

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [2.]])

In [15]:
y_test[:10]

array([[1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [4]:
from extract import load_set
import pandas as pd
import numpy as np

data_dir = './data/prepared/august25screenfixed/'
stock = 'RRR'
suffix = ''
df = load_set(stock, data_dir, suffix)    
    
df = df.dropna(axis=0)

e = df.shape[0]
l = e - 4000 - 59

y_ = df.iloc[l:e-59]['decision']

dt = df.transpose()

for j, i in enumerate(range(l, e-59)):
    display(dt.iloc[:, i:i+60])
    break

,4084,4083,4082,4081,4080,4079,4078,4077,4076,4075,...,4034,4033,4032,4031,4030,4029,4028,4027,4026,4025
open,14.1,14.1,14.12,14.13,14.16,14.1666,14.17,14.16,14.1703,14.09,...,14.02,14.0513,14.22,14.33,14.5,14.51,14.505,14.575,14.585,14.63
high,14.12,14.13,14.12,14.15,14.2,14.18,14.18,14.17,14.2,14.14,...,14.15,14.1,14.2243,14.335,14.5,14.555,14.52,14.575,14.66,14.67
low,14.09,14.09,14.095,14.11,14.12,14.1449,14.15,14.1499,14.165,14.09,...,14.02,14.05,14.029,14.17,14.33,14.49,14.47,14.52,14.56,14.56
close,14.12,14.09,14.11,14.12,14.12,14.18,14.17,14.17,14.17,14.135,...,14.14,14.07,14.029,14.19,14.33,14.5,14.505,14.52,14.575,14.6
volume,2400,3502,7204,5459,12245,7635,3874,3790,3047,1373,...,11400,4062,3950,19368,7300,3608,11813,3425,22165,18918
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
close_xsinxcosx2,1.29604e-06,-1.04494e-06,-1.28058e-07,-0,-4.89959e-07,-3.04935e-08,-0,-0,2.80321e-06,-2.96302e-07,...,6.2469e-06,-9.47354e-06,1.15471e-05,7.12522e-06,-2.61118e-05,5.94785e-07,-1.11564e-06,-1.32737e-05,1.00653e-06,2.70799e-06
close_xtanx,0.00400648,-0.00032803,0.000174534,-0,-0.0172016,0.000142469,-0,0,0.00309142,-0.000664253,...,-0.00247104,0.00467061,0.131381,0.118296,0.0289888,-0.000270486,0.0318026,0.00480437,0.00123597,-0.000184497
close_xtanx2,0.00557486,-0.000517237,8.3285e-05,-0,-0.0357574,0.000232094,0,0,0.00396838,0.000755574,...,-0.00219489,0.00495615,0.12983,0.116848,0.0153533,-0.000469115,0.00223592,0.00208403,0.000396816,-0.00244977
decision,0,0,0,0,0,2,2,2,2,0,...,2,0,0,2,2,2,2,2,2,2
